In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import sys
#sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
import glob
import json
import torch.nn as nn

import torch
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms



from tqdm import trange
from torch.nn import init
import torch.optim as optim


from torch.utils.data import DataLoader,Dataset
import cv2, sys, copy, scipy
from torch.utils.tensorboard import SummaryWriter

import torch.nn.functional as F

os.chdir("/content/drive/My Drive/Colab Notebooks/data_ntechlab")

data_dir = "/content/drive/My Drive/Colab Notebooks/data_ntechlab"

In [ ]:
## Загрузка и преобразование данных. Разбивка датасета в пропорции train_idx/test_idx (0.7/0.3)

In [ ]:
data_transforms = transforms.Compose(
    [transforms.Resize(226),
     transforms.CenterCrop(224),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

whole_set = ImageFolder(data_dir, transform = data_transforms) 


test_idx = list(np.random.choice(range(len(whole_set)), 30000, replace=False))
set_test = test_idx.copy()
train_idx = list(set(range(len(whole_set))) - set(set_test))


trainset_subset = torch.utils.data.Subset(whole_set, train_idx)
testset_subset = torch.utils.data.Subset(whole_set, test_idx)

trainloader = torch.utils.data.DataLoader(trainset_subset, batch_size=250, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset_subset, batch_size=250, shuffle=True, num_workers=2)

In [ ]:
def train(model, epoches, criterion, optimizer, trainloader, testloader, device, exp_name=None, scheduler=None):
    
    best_model_wts = copy.deepcopy(model.state_dict())

    if exp_name:
        writer = SummaryWriter('./logs_weeklearns/{}'.format(exp_name))
    else:
        writer = SummaryWriter('./logs_weeklearns/{}'.format('single_ep'))
        
    bestval = {'bestvalacc': 0, 'epoch': None, 'trainacc@Bval': None, 'iter': 0}
    for e in (range(1, epoches+1)):
        
        model.train()
        meanloss = {'counter':0, 'correct':0, 'total':0}
        for X, y in (trainloader):
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            pred = model(X)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            bestval['iter'] +=1
            meanloss['total'] += len(y)
            meanloss['correct'] += (torch.argmax(pred, dim=1) == y).sum().item()
            writer.add_scalar("train_loss", loss.item(), global_step=bestval['iter'])


        writer.add_scalar("train_epoch_Accuracy", meanloss['correct']/len(trainloader.dataset), global_step=e)
        print('Epoch:{} |train_accuracy:{}'.format(e, meanloss['correct']/meanloss['total']))
        model.eval()
        meanlossval = {'loss':0, 'counter':0, 'correct':0,'total':0}
        for X, y in testloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            meanlossval['loss'] += criterion(pred, y).item()
            meanlossval['total'] += len(y)
            meanlossval['correct'] += (torch.argmax(pred, dim=1) == y).sum().item()

        writer.add_scalar("test_epoch_Accuracy", meanlossval['correct']/len(testloader.dataset), global_step=e)
        print('Epoch:{} |test_accuracy:{}'.format(e, meanlossval['correct']/meanlossval['total']))
        print('--'*50)
        if scheduler:
            scheduler.step()
            
        if meanlossval['correct']/meanlossval['total'] > bestval['bestvalacc']:
            bestval['bestvalacc'] = meanlossval['correct']/meanlossval['total']
            bestval['trainacc@Bval'] = meanloss['correct']/meanloss['total']
            bestval['epoch'] = e
            best_model_wts = copy.deepcopy(model.state_dict())
            
    print(bestval)
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# Архитектура модели

In [ ]:
class CNNClassifier(nn.Module):

    def __init__(self, channels=3 , num_classes=2):
        super(CNNClassifier, self).__init__()
        
        self.conv1 = torch.nn.Conv2d(channels, 32, kernel_size=7, stride=3, padding=1)
        self.conv2 = torch.nn.Conv2d(32, 32, kernel_size=5, stride=1, padding=1)
        self.conv3 = torch.nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1)
        self.conv4 = torch.nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.drop_layer_conv = nn.Dropout(p=0.25)
        self.drop_layer_dense = nn.Dropout(p=0.5)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = torch.nn.Linear(64 * 9 * 9, 512)
        self.fc2 = torch.nn.Linear(512, num_classes)

    def forward(self, x):
      # Block 1
      x = F.relu(self.conv1(x))
      x = F.relu(self.conv2(x))
      x = self.pool(x)
      x = self.drop_layer_conv(x)

      # Block 2
      x = F.relu(self.conv3(x))
      x = F.relu(self.conv4(x))
      x = self.pool(x)
      x = self.drop_layer_conv(x)
      x = x.view(-1, 64 * 9 * 9)
      x = F.relu(self.fc1(x))
      x = self.drop_layer_dense(x)
      x = self.fc2(x)
      return x

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [ ]:
from datetime import datetime
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

model = CNNClassifier()
model.to(device)
model.apply(weights_init)
epoches = 10
scheduler = None

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam( model.parameters(),
                              lr=1e-4,
                              weight_decay=1e-6)
exp_name = datetime.now().isoformat()

In [ ]:
best_model=train(model, epoches, criterion, optimizer, trainloader, testloader, device, exp_name, scheduler)

Epoch:1 |train_accuracy:0.7818137668014112
Epoch:1 |test_accuracy:0.8650333333333333
----------------------------------------------------------------------------------------------------
Epoch:2 |train_accuracy:0.8825008213229728
Epoch:2 |test_accuracy:0.9120666666666667
----------------------------------------------------------------------------------------------------
Epoch:3 |train_accuracy:0.9095544858518191
Epoch:3 |test_accuracy:0.9172
----------------------------------------------------------------------------------------------------
Epoch:4 |train_accuracy:0.9219243240154837
Epoch:4 |test_accuracy:0.9331
----------------------------------------------------------------------------------------------------
Epoch:5 |train_accuracy:0.93173734805525
Epoch:5 |test_accuracy:0.9222
----------------------------------------------------------------------------------------------------
Epoch:6 |train_accuracy:0.9356082789355654
Epoch:6 |test_accuracy:0.8922333333333333
-----------------------

In [ ]:
# Сохранение модели с лучшими параметрами

In [ ]:
model_save_name = 'Best_mobnet_all.pth'
path = F"/content/drive/My Drive/Colab Notebooks/{model_save_name}" 
torch.save(best_model.state_dict(), path, _use_new_zipfile_serialization=False)

NameError: ignored